# Predict Future Sales

Second part of feauture enginering and ML modeling - testing advanced models and advanced features

kaggle link : https://www.kaggle.com/c/competitive-data-science-predict-future-sales/overview

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import seaborn as sns; sns.set()
import random
from dateutil.relativedelta import relativedelta
import os

In [2]:
from my_functions import merge_basic, convert_silver, completion_semi_gold, scaler, inversed_scale, consolidation_prediction, plot_prediction, preparing_to_predict, plot_metrics

In [3]:
from especial_functions import balance_items_test, get_full_lags, feature_silver, integration_new_features_map
from especial_functions import features_rows_train_silver, exploiting_words, get_maps_from, augmentation_reduction

In [4]:
from sklearn.preprocessing import MinMaxScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error

In [5]:
#from sklearn_extra.cluster import KMedoids
from sklearn.cluster import KMeans

### Data

In [6]:
daily_train = pd.read_csv('sales_train.csv')
items = pd.read_csv('items.csv')
shops = pd.read_csv('shops.csv')
categories = pd.read_csv('item_categories.csv')
sample_submission = pd.read_csv('sample_submission.csv')
test = pd.read_csv('test.csv')

### Data modeling

In [7]:
train_merge = merge_basic(daily_train, test,'left')

In [8]:
train_merge.head(5)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,item_category_name,comp1_x,comp2_x,map_comp1categ,map_comp2categ,shop_name,comp1_y,comp2_y,map_comp1shop,map_comp2shop,ID,SaleDate
0,02.01.2013,0,59,22154,999.00,1.0,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray,Кино,Blu-Ray,11,19,"Ярославль ТЦ ""Альтаир""",Ярославль,Альтаир,31,45,150983.0,2013-01-02
1,03.01.2013,0,25,2552,899.00,1.0,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,Музыка,Винил,13,39,"Москва ТРК ""Атриум""",Москва,Атриум,14,18,NaN,2013-01-03
2,05.01.2013,0,25,2552,899.00,-1.0,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,Музыка,Винил,13,39,"Москва ТРК ""Атриум""",Москва,Атриум,14,18,NaN,2013-01-05
3,06.01.2013,0,25,2554,1709.05,1.0,DEEP PURPLE Who Do You Think We Are LP,58,Музыка - Винил,Музыка,Винил,13,39,"Москва ТРК ""Атриум""",Москва,Атриум,14,18,NaN,2013-01-06
4,15.01.2013,0,25,2555,1099.00,1.0,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства,Музыка,CD фирменного производства,13,37,"Москва ТРК ""Атриум""",Москва,Атриум,14,18,NaN,2013-01-15


In [9]:
wholedata_silver = convert_silver(train_merge)

In [10]:
wholedata_silver.head()

,SaleDateMY,date_block_num,shop_id,item_id,item_name,item_category_id,shop_name,item_category_name,map_comp1categ,map_comp2categ,map_comp1shop,map_comp2shop,InTest,item_cnt_day,item_price,shopitem
0,2013-01-01,0,0,32,1+1,40,"!Якутск Орджоникидзе, 56 фран",Кино - DVD,11,22,0,0,0,6.0,221.0,0-32
1,2013-01-01,0,0,33,1+1 (BD),37,"!Якутск Орджоникидзе, 56 фран",Кино - Blu-Ray,11,19,0,0,0,3.0,347.0,0-33
2,2013-01-01,0,0,35,10 ЛЕТ СПУСТЯ,40,"!Якутск Орджоникидзе, 56 фран",Кино - DVD,11,22,0,0,0,1.0,247.0,0-35
3,2013-01-01,0,0,43,100 МИЛЛИОНОВ ЕВРО,40,"!Якутск Орджоникидзе, 56 фран",Кино - DVD,11,22,0,0,0,1.0,221.0,0-43
4,2013-01-01,0,0,51,100 лучших произведений классики (mp3-CD) (Dig...,57,"!Якутск Орджоникидзе, 56 фран",Музыка - MP3,13,38,0,0,0,2.0,130.0,0-51


In [11]:
wholedata_silver.shape

(1609124, 16)

<b> Splitting strategy:

* time series expanding window of one period, Take 7 months
* over the first fold, make the functions and features for modeling and prediction (avoid leakage)
* lags and seasons are features
* use rmse as metric
* consider the outputs of the EDA
* goal, predict november or yyyy-11-01

#### The validation data

In [12]:
wholedata_silver.SaleDateMY.max()

Timestamp('2015-10-01 00:00:00')

In [13]:
test_dates = [datetime.datetime(2014, 11, 1), datetime.datetime(2015, 4, 1), datetime.datetime(2015, 5, 1), datetime.datetime(2015, 6, 1), datetime.datetime(2015, 7, 1), datetime.datetime(2015, 8, 1),
datetime.datetime(2015, 9, 1), datetime.datetime(2015, 10, 1),]

#### hold-out Train

In [14]:
#gato
train_big_window = wholedata_silver
train_big_full = completion_semi_gold(train_big_window)

In [15]:
date_val = datetime.datetime(2014, 10, 1)

In [16]:
train_full = train_big_full[train_big_full.Date < date_val ].copy()
val_window = train_big_full[train_big_full.Date == date_val ].copy()

In [17]:
train_full.head()

,InTest,shop_id,item_id,item_name,shopitem,item_category_id,shop_name,item_category_name,map_comp1categ,map_comp2categ,map_comp1shop,map_comp2shop,Date,Sale,item_price,Order,MiguelId
0,0,0,30,007: КООРДИНАТЫ «СКАЙФОЛЛ»,0-30,40,"!Якутск Орджоникидзе, 56 фран",Кино - DVD,11,22,0,0,2013-01-01,0.0,0.0,1,0-30-2013-01-01
1,0,0,31,007: КООРДИНАТЫ «СКАЙФОЛЛ» (BD),0-31,37,"!Якутск Орджоникидзе, 56 фран",Кино - Blu-Ray,11,19,0,0,2013-01-01,0.0,0.0,1,0-31-2013-01-01
2,0,0,32,1+1,0-32,40,"!Якутск Орджоникидзе, 56 фран",Кино - DVD,11,22,0,0,2013-01-01,6.0,221.0,1,0-32-2013-01-01
3,0,0,33,1+1 (BD),0-33,37,"!Якутск Орджоникидзе, 56 фран",Кино - Blu-Ray,11,19,0,0,2013-01-01,3.0,347.0,1,0-33-2013-01-01
4,0,0,35,10 ЛЕТ СПУСТЯ,0-35,40,"!Якутск Орджоникидзе, 56 фран",Кино - DVD,11,22,0,0,2013-01-01,1.0,247.0,1,0-35-2013-01-01


In [18]:
train_full.Date.max(), train_full.Date.min()

(Timestamp('2014-09-01 00:00:00'), Timestamp('2013-01-01 00:00:00'))

In [19]:
train_full.shape

(8906604, 17)

In [20]:
train_full.groupby('InTest').agg(shopitems = ('shopitem','nunique'))

,shopitems
InTest,
0,312720
1,111404


### Exploration of the features

In [21]:
%%time
features_dictionary = get_full_lags(whole = train_full, date_to_take = date_val) ## for train but result easy in val
item_explo_names = exploiting_words(train_full)
#feature_cluster = get_clusters(data = train_full, n_clusters_features = 5, date_to_take = date_val )
full_features_dictionary = {**features_dictionary,
                            **item_explo_names
                            #**feature_cluster
                           }

Wall time: 1min 19s


In [22]:
%%time
data_train_selected = balance_items_test(train_full, date_to_take = date_val, percentage_test = 0.30, smalls_percent = 0.5, gen_sample = 0.25) ## for train
train_feature_tocomplete = feature_silver(data_train_selected, date_to_take = date_val, Train = True) ## for both

Wall time: 19.8 s


In [23]:
train_feature = integration_new_features_map(data = train_feature_tocomplete, dict_feature = full_features_dictionary)  ## for both not prob

In [24]:
train_feature.columns 

Index(['InTest', 'shop_id', 'item_id', 'item_name', 'shopitem',
       'item_category_id', 'shop_name', 'item_category_name', 'map_comp1categ',
       'map_comp2categ', 'map_comp1shop', 'map_comp2shop', 'Date', 'Sale',
       'item_price', 'Order', 'MiguelId', 'SaleL1M', 'L0M_L1M', 'L1M_L2M',
       'ItemIdPart1', 'ItemIdPart2', 'CategIdItem3A', 'CategIdItem3B', 'month',
       'monthToPredict', 'Year', 'Quarter', 'OrderGot', 'countItemPossitives',
       'countItemZero', 'L1maxLev1Item', 'L2maxLev1Item', 'L3maxLev1Item',
       'L4maxLev1Item', 'L5maxLev1Item', 'CategoryL1Item',
       'countItemShopPossitives', 'countItemShopZero', 'L1maxLev1ItemShop',
       'L2maxLev1ItemShop', 'L3maxLev1ItemShop', 'L4maxLev1ItemShop',
       'L5maxLev1ItemShop', 'lenstr', 'NWords', 'Word0', 'Word_1',
       'L1maxLev1Shop', 'meanLaggedVars', 'item_idLastMonth1',
       'item_idLastMonth2', 'item_idLastMonth3', 'item_idLastMonth4',
       'item_idLastMonth5', 'item_idLastOrder1', 'item_idLastOrder2

In [25]:
to_drop_columns = ['Date']
ids_columns = ['item_category_id',
       'ItemIdPart1','ItemIdPart2', 'CategIdItem3A', 'CategIdItem3B'] #'item_id'
numericals = [ 'Quarter', 'OrderGot', 'countItemPossitives',
       'countItemZero', 'L1maxLev1Item', 'L2maxLev1Item', 'L3maxLev1Item',
       'L4maxLev1Item', 'L5maxLev1Item',
       'countItemShopPossitives', 'countItemShopZero', 'L1maxLev1ItemShop',
       'L2maxLev1ItemShop', 'L3maxLev1ItemShop', 'L4maxLev1ItemShop',
       'L5maxLev1ItemShop', 'lenstr', 'NWords', 'Word0', 'Word_1',
       'L1maxLev1Shop', 'meanLaggedVars', 'item_idLastMonth1',
       'item_idLastMonth2', 'item_idLastMonth3', 'item_idLastMonth4',
       'item_idLastMonth5', 'item_idLastOrder1', 'item_idLastOrder2',
       'item_idLastOrder3', 'item_idLastOrder4', 'item_idLastOrder5',
       'meanL1Price', 'SpaceOrder'] + ids_columns

categoricals = ['SpaceOrderActivator', 'CategoryL1Item']
my_features = to_drop_columns + numericals + categoricals
my_target = 'Sale'
len(my_features) - 3

39

In [26]:
train_full.Sale.max()

20.0

### Production of folds for cross validation

In [27]:
sample_degree = 0.75
orderdate = 0
limit = 6
maps = ['Starts','Focus_item_features','Focus_item_shop_features','item_name_explo','Focus_shop_features',
       'item previous var', 'lasthmonth_item', 'lasthOrder_item','meanL1Price_item',]
len(maps)

9

In [28]:
def date_generator(Date_list, limit = 5):
    date_list = list()
    for date_val in Date_list:
        dates_back = 3
        ### Date cqpture ####################
        date_max = date_val - relativedelta(months=1)
        months_to_get = [4,5,6,10,11]
        minyear = 2014
        min_date = date_max - relativedelta(months=dates_back)

        year_i = date_max.year

        dates_result = list()

        while year_i >= minyear:
            for monthi in months_to_get:
                dates_result.append(datetime.datetime(year_i, monthi, 1))
                dates_result = [x for x in dates_result if x <= date_max]
            year_i = year_i - 1

        dates_result = dates_result + [date_max - relativedelta(months=i) for i in range(0,dates_back +1)]
        dates_result = list(set(dates_result))
        for daten in dates_result:
            date_list.append(daten)
    date_list = list(set(date_list))  
    date_list.sort()
    return date_list + [datetime.datetime(2015, 10, 1, 0, 0), datetime.datetime(2015, 11, 1, 0, 0) ]

In [29]:
dates_vector_map =  date_generator(Date_list = test_dates, limit = limit)

In [34]:
#gato
for datex in dates_vector_map:
    datex_str = datex.strftime('%Y-%m-%d')
    train_full = train_big_full[train_big_full.Date < datex].copy()
    features_dictionary = get_full_lags(whole = train_full, date_to_take = datex) ## for train but result easy in val
    item_explo_names = exploiting_words(train_full)
    #feature_cluster = get_clusters(data = train_full, n_clusters_features = 5, date_to_take = date_val )
    full_features_dictionary = {**features_dictionary,
                                **item_explo_names
                                #**feature_cluster
                               }
    
    outdir = f'./generated_datasets/maps/data_{datex_str}/'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
        
    for key in full_features_dictionary.keys():
        mapx = full_features_dictionary[key]
        map_name = os.path.join(outdir, f'{key}.csv') 
        mapx.to_csv(path_or_buf = map_name,header=True, index=False)
    del train_full, features_dictionary, full_features_dictionary, item_explo_names
    print(f'the maps for the date {datex_str}')

the maps for the date 2014-04-01
the maps for the date 2014-05-01
the maps for the date 2014-06-01
the maps for the date 2014-07-01
the maps for the date 2014-08-01
the maps for the date 2014-09-01
the maps for the date 2014-10-01
the maps for the date 2014-11-01
the maps for the date 2014-12-01
the maps for the date 2015-01-01
the maps for the date 2015-02-01
the maps for the date 2015-03-01
the maps for the date 2015-04-01
the maps for the date 2015-05-01
the maps for the date 2015-06-01
the maps for the date 2015-07-01
the maps for the date 2015-08-01
the maps for the date 2015-09-01
the maps for the date 2015-10-01
the maps for the date 2015-11-01


In [30]:
def get_maps_from(datex_str, maps):
    features_mapping = dict()
    for key in maps:
        mapsx = pd.read_csv(f'generated_datasets/maps/data_{datex_str}/{key}.csv')
        if 'Date' in mapsx.columns:
            mapsx['Date'] = pd.to_datetime(mapsx['Date'])
        features_mapping[key] = mapsx
    return features_mapping

In [32]:
#for date_val in test_dates:
for date_val in [datetime.datetime(2015, 11, 1, 0, 0)]:
    
    date_val_str = date_val.strftime('%Y-%m-%d')
    train_full = train_big_full[train_big_full.Date < date_val ].copy()
    val_window = train_big_full[train_big_full.Date == date_val ].copy()
    
    ### splits
    ### conventional for regular time series
    dates_back = 3
    ### Date cqpture ####################
    date_max = date_val - relativedelta(months=1)
    months_to_get = [4,5,6,10,11]
    minyear = 2014
    min_date = date_max - relativedelta(months=dates_back)

    year_i = date_max.year

    dates_result = list()

    while year_i >= minyear:
        for monthi in months_to_get:
            dates_result.append(datetime.datetime(year_i, monthi, 1))
            dates_result = [x for x in dates_result if x <= date_max]
        year_i = year_i - 1

    dates_result = dates_result + [date_max - relativedelta(months=i) for i in range(0,dates_back +1)]
    dates_result = list(set(dates_result))
    dates_result.sort()
    dates_result = dates_result[::-1]
    list_dfs = list()
       
    for datex,i in zip(dates_result[0:limit], range(len(dates_result[0:limit]))):
        datex_plus1 = datex + relativedelta(months=1)
        datex_plus1_str = datex_plus1.strftime('%Y-%m-%d')
        full_features_dictionary = get_maps_from(datex_str = datex_plus1_str, maps = maps)
        
        
        data_train_selected = balance_items_test(data = train_full, date_to_take = datex_plus1, percentage_test = 0.30, smalls_percent = 0.5, gen_sample = sample_degree) ## for train
        train_feature_tocomplete = feature_silver(data_train_selected, date_to_take = datex_plus1, Train = True) ## for both
        train_feature = integration_new_features_map(data = train_feature_tocomplete, dict_feature = full_features_dictionary)  ## for both not prob
        df = train_feature[train_feature.Date == datex]

        list_dfs.append(df)
        del df, train_feature, train_feature_tocomplete, data_train_selected
       
    train_selection = pd.concat(list_dfs).fillna(0).reset_index(drop = True)
    
    full_features_dictionary = get_maps_from(datex_str = date_val_str, maps = maps)
    
    full_valandtrain_selected = val_window
    valandtrain_feature = feature_silver(full_valandtrain_selected, date_to_take = date_val, Train = False)
    valandtrain_feature = integration_new_features_map(data = valandtrain_feature, dict_feature = full_features_dictionary, filter_ceros = True) 
    val_selection = features_rows_train_silver(data = valandtrain_feature, features = my_features, target = my_target, validation = date_val, additional_val_cols = ['shop_id', 'item_id'])   
    
    
    ### Saving DataFrames
    outdir = f'./generated_datasets/data_{date_val_str}/'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
       
    train_name = os.path.join(outdir, 'train_selection.csv') 
    train_selection.to_csv(path_or_buf = train_name,header=True, index=False)
    
    val_name = os.path.join(outdir, 'val_selection.csv') 
    val_selection.to_csv(path_or_buf = val_name,header=True, index=False)
    
    del train_selection, full_valandtrain_selected, valandtrain_feature, val_selection
    
    print(f'the datasets for the date {date_val_str}')

IllegalMonthError: bad month number nan; must be 1-12

In [33]:
#in case jojo
outdir = f'./generated_datasets/data_{date_val_str}/'
if not os.path.exists(outdir):
    os.mkdir(outdir)

train_name = os.path.join(outdir, 'train_selection.csv') 
train_selection.to_csv(path_or_buf = train_name,header=True, index=False)
print(f'the datasets for the date {date_val_str}')

the datasets for the date 2015-11-01
